In [66]:
import numpy as np
import pandas as pd
import scipy



from Dissertation import RandomGenerator, convert_to_int, get_order_index
rg = RandomGenerator(5, 100, order_by = 'var', use_dask = False)

dataset is ordered by var


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [67]:
df = rg()

#### Brief look at the data

In [68]:
df.head()

,X_0,X_1,X_2,X_3,X_4,X_1X_4,X_0X_2X_4,X_0X_1X_3,X_0X_1X_2X_4,y
0,0,1,0,1,1,1,0,0,0,4.783196
1,1,1,1,1,0,0,0,1,0,4.199899
2,0,1,0,1,0,0,0,0,0,9.655961
3,0,0,1,0,0,0,0,0,0,-2.625972
4,1,0,0,0,0,0,0,0,0,3.217509


### How to find the full model

In [69]:
df['X_int'] = df.iloc[:,:5].apply(convert_to_int, axis = 1)

In [70]:
prediction = df.groupby('X_int')['y'].agg(np.mean)
df['pred'] = df.X_int.apply(lambda x: prediction[x])

In [71]:
prediction

X_int
0     -2.778951
2     -1.672351
4      0.525211
6      2.286356
7     -7.444686
8      8.728226
10     9.655961
11     4.183016
12     7.620110
14     4.016774
15     3.947661
16     3.217509
17     0.871730
18     8.766128
19     4.500237
20     6.250312
21     7.238610
22     5.840614
23     0.310708
24     6.922149
25     8.492836
26     7.006050
27     2.374025
28     8.367708
29    14.967918
30     3.369157
31    11.328444
Name: y, dtype: float64

In [76]:
SSTO = np.sum((df.y - df.y.mean())**2)
SSE = np.sum((df.y - df.pred)**2)
R_sqf = (SSTO - SSE)/SSTO

In [77]:
R_sqf

0.4607977754866765

### Aggregation / P-way interaction

In [80]:
aggregation = df.iloc[:, :5].apply(np.sum, axis = 1)

In [81]:
aggregation

0     3
1     4
2     2
3     1
4     1
     ..
95    0
96    4
97    4
98    2
99    1
Length: 100, dtype: int64

In [82]:
# Treating aggregation as a categorical variable

In [87]:
pred2 = df.groupby(aggregation)['y'].agg(np.mean)

In [89]:
df['pred2'] = aggregation.apply(lambda x: pred2[x])

In [92]:
SSTO = np.sum((df.y - df.y.mean())**2)
SSE = np.sum((df.y - df.pred2)**2)
R_sqa = (SSTO - SSE)/SSTO

In [94]:
(R_sqf-R_sqa)/(1-R_sqf) #tau

0.6619202525553443

In [99]:
# Treating aggregation as numeric variable
from sklearn.linear_model import LinearRegression
reg = LinearRegression()

In [104]:
reg.fit(aggregation.to_numpy().reshape(-1,1), df.y)

LinearRegression()

In [108]:
df['pred3'] = reg.predict(aggregation.to_numpy().reshape(-1,1))

In [109]:
SSTO = np.sum((df.y - df.y.mean())**2)
SSE = np.sum((df.y - df.pred3)**2)
R_sqa2 = (SSTO - SSE)/SSTO
(R_sqf-R_sqa2)/(1-R_sqf) #tau

0.8138574115951377

### p-way interaction

In [113]:
df['pint'] = df.iloc[:, :5].apply(lambda x: 1 if np.sum(x) > 0 else 0, axis = 1)

In [116]:
reg.fit(df.pint.to_numpy().reshape(-1,1), df.y)

LinearRegression()

In [118]:
df['pred4'] = reg.predict(df.pint.to_numpy().reshape(-1,1))

In [120]:
SSTO = np.sum((df.y - df.y.mean())**2)
SSE = np.sum((df.y - df.pred4)**2)
R_sqpint = (SSTO - SSE)/SSTO
(R_sqf-R_sqpint)/(1-R_sqf) #tau

0.8153847520892585